We will use a famous library named TensorFlow (which is now available in version 2+) and in particular one of its subpackages, Keras, together with some utilities libraries like: 

- Matplotlib (for visualization);
- Numpy (to work with arrays);

## GPU Runtime

Neural Network training requires high parallel computation. 


In [1]:
# Utilities
import numpy as np
import matplotlib.pyplot as plt

# Deep Learning
import tensorflow as tf
from tensorflow import keras

# Check tensorflow version (must be >2!)
print(tf.__version__)

2024-08-11 10:56:46.899616: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


2.4.1


## Read a dataset

To train a Neural Network model, we will need to load in memory a dataset. You can load it in lots of ways, depending on the time of data that you need.

we will use built-in data on keras. In particoular, we are interested in:

- Fashion-MNIST: It is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. 

we will download the dataset locally and experiment with it.
You can visualize the data [here](https://knowyourdata-tfds.withgoogle.com/#tab=STATS&dataset=fashion_mnist).

In [2]:
# Import keras dataset Fashion Mnist
from tensorflow.keras.datasets import fashion_mnist

# Load the data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Check data dimensionality
print(f"Training set dimension: Input {x_train.shape}, Output {y_train.shape}")
print(f"Test set dimension: Input {x_test.shape}, Output {y_test.shape}")

Training set dimension: Input (60000, 28, 28), Output (60000,)
Test set dimension: Input (10000, 28, 28), Output (10000,)


We would like our input data to lies in the interval $[0, 1]$. If our data does not lies in this interval, we can transform it as:

$$
x' = \frac{x - x_{min}}{x_{max}-x_{min}}
$$

Where $x_{min} = \min(x)$, $x_{max} = \max(x)$. Note that $x'$ always lies in the interval $[0, 1]$.

In [3]:
#Normalization function
normalize_data = (lambda X: ((X - X.min()) / (X.max() - X.min())))

This operation is called normalization(min-max feature scaling) and allow us to work easily with the data, speeding up the computation and in some case improving even the results of the training.

In [4]:
print(f"Input (train) data lies in the interval [{x_train.min()}, {x_train.max()}]")
print(f"Input (test) data lies in the interval [{x_test.min()}, {x_test.max()}]")
    
x_train = normalize_data(x_train)
x_test = normalize_data(x_test)

# Check the interval after normalization
print("\n")
print(f"Input (train) data lies in the interval [{x_train.min()}, {x_train.max()}]")
print(f"Input (test) data lies in the interval [{x_test.min()}, {x_test.max()}]")

Input (train) data lies in the interval [0, 255]
Input (test) data lies in the interval [0, 255]


Input (train) data lies in the interval [0.0, 1.0]
Input (test) data lies in the interval [0.0, 1.0]


In [5]:
print(f"y[0]: {y_train[0]}")

# One hot encode the output variables (needed to compute the output)
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

n_classes = len(y_train[0])

print(f"y[0] after the one-hot encoding: {y_train[0]}")

y[0]: 9
y[0] after the one-hot encoding: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
